In [2]:
import pandas as pd
import pathlib
import numpy as np

## Load Data

In [3]:
raw_data_dir = pathlib.Path("../../data/raw/ms_work/")

trans_res_path = raw_data_dir.joinpath('BSA_Oval_2023_06_12_Transition Results.csv')
trans_res_df_raw = pd.read_csv(trans_res_path)
trans_res_df_raw.columns = trans_res_df_raw.columns.map(str.lower)
trans_res_df_raw.head()

,peptide,protein,replicate,precursor mz,precursor charge,product mz,product charge,fragment ion,retention time,area,background,peak rank
0,GGLEPINFQTAADQAR,sp|P01012|OVAL_CHICK,Soroush_C18_SDS_D2_column12_2uL_13,844.42355,2,844.42355,2,precursor,46.54,4396397568,35182616,1
1,GGLEPINFQTAADQAR,sp|P01012|OVAL_CHICK,Soroush_C18_SDC_column12_2uL_22,844.42355,2,844.42355,2,precursor,45.45,717893184,0,1
2,GGLEPINFQTAADQAR,sp|P01012|OVAL_CHICK,Soroush_C18_SDC_D2_column12_2uL_11,844.42355,2,844.42355,2,precursor,45.85,609600704,0,1
3,GGLEPINFQTAADQAR,sp|P01012|OVAL_CHICK,Soroush_C18_SDC_D3_column12_2uL_22,844.42355,2,844.42355,2,precursor,46.14,653753856,0,1
4,GGLEPINFQTAADQAR,sp|P01012|OVAL_CHICK,Soroush_C18_SDS_column12_2uL_21,844.42355,2,844.42355,2,precursor,45.79,4406924288,26830026,1


In [4]:

trans_res_df_raw.peptide.unique()

array(['GGLEPINFQTAADQAR', 'AEFVEVTK', 'YICDNQDTISSK'], dtype=object)

## Compute Area ratio

In [5]:
# def grp_area_ratio(grouped_df:pd.DataFrame):
    
#     """compute the ratio of areas with higer precursor mz value as numerator."""

#     df = grouped_df[['precursor mz', 'area']].copy()
#     df = df.sort_values(by='precursor mz', ascending=True)
#     area_ratio = df['area'].iloc[1] / df['area'].iloc[0]

#     return area_ratio

def grp_area_ratio_n_area_cols(grouped_df:pd.DataFrame):
    
    """compute the ratio of areas with higer precursor mz value as numerator."""

    df = grouped_df[['precursor mz', 'area']].copy()
    df = df.sort_values(by='precursor mz', ascending=True)
    area_ratio = df['area'].iloc[1] / df['area'].iloc[0]
    area_min, area_max  =  df['area'].iloc[0], df['area'].iloc[1]

    col_vals = {'area_ratio': area_ratio, 'area_min':area_min, 'area_max':area_max}

    return pd.Series(col_vals)

cols_to_group_by = ['peptide', 'protein', 'replicate', 'product charge', 'fragment ion']
# df_area_ratio = trans_res_df_raw.groupby(cols_to_group_by).apply(grp_area_ratio).reset_index(name='area_ratio')
df_area_ratio = trans_res_df_raw.groupby(cols_to_group_by).apply(grp_area_ratio_n_area_cols).reset_index()


# df_area_ratio = df_area_ratio[df_area_ratio['fragment ion'] == 'precursor']  # select only precursor
print(df_area_ratio.shape)
df_area_ratio.head()

(270, 8)


,peptide,protein,replicate,product charge,fragment ion,area_ratio,area_min,area_max
0,AEFVEVTK,sp|P02769|ALBU_BOVIN,Soroush_C18_SDC_D2_column12_2uL_11,1,b2,4.785271,425566400.0,2.036451e+09
1,AEFVEVTK,sp|P02769|ALBU_BOVIN,Soroush_C18_SDC_D2_column12_2uL_11,1,y2,5.214290,52236184.0,2.723746e+08
2,AEFVEVTK,sp|P02769|ALBU_BOVIN,Soroush_C18_SDC_D2_column12_2uL_11,1,y4,4.992701,92338456.0,4.610183e+08
3,AEFVEVTK,sp|P02769|ALBU_BOVIN,Soroush_C18_SDC_D2_column12_2uL_11,1,y5,5.242521,117168584.0,6.142588e+08
4,AEFVEVTK,sp|P02769|ALBU_BOVIN,Soroush_C18_SDC_D2_column12_2uL_11,1,y6,5.069243,377321984.0,1.912737e+09


## Get peptide Dilution Conc.

In [6]:
df_peptide_dilution_conc = pd.read_csv(raw_data_dir.joinpath('peptide_dilution_conc.csv'),
                                    header=None)
df_peptide_dilution_conc.columns = ['peptide', 'replicate', 'fragment ion', 'area_ratio', 'heavy_conc']
df_peptide_dilution_conc.dropna(axis=0, inplace=True)
df_peptide_dilution_conc

def get_dilution_from_replicate(rep_string:str):
    """Get the dilution symbol from the replicate name."""

    str_to_check = rep_string.split('_')[3]
    if str_to_check.startswith('col'):
        dilution = 'D0'
    else:
        dilution = str_to_check

    return dilution

def get_peptide_dilution_comb(row):
    """function to create a new column from peptide and dilution name."""
    peptide = row['peptide']
    replicate  = row['replicate']
    dilution = get_dilution_from_replicate(rep_string=replicate)

    return peptide + '_' + dilution

peptide_dilution_name_col = 'peptide_dilution_name'
df_peptide_dilution_conc[peptide_dilution_name_col] = df_peptide_dilution_conc.apply(get_peptide_dilution_comb, axis=1)
df_peptide_dilution_conc.drop_duplicates(subset=peptide_dilution_name_col, inplace=True)
df_peptide_dilution_conc.head(2)


,peptide,replicate,fragment ion,area_ratio,heavy_conc,peptide_dilution_name
0,GGLEPINFQTAADQAR,Soroush_C18_SDS_column12_2uL_21,precursor,0.004588,2.208000,GGLEPINFQTAADQAR_D0
1,GGLEPINFQTAADQAR,Soroush_C18_SDS_D3_column12_2uL_21,precursor,0.253738,15.771428,GGLEPINFQTAADQAR_D3


### Merge data

In [7]:
df_area_ratio[peptide_dilution_name_col] = df_area_ratio.apply(get_peptide_dilution_comb, axis=1)



df_area_ratio_conc = pd.merge(df_area_ratio, right=df_peptide_dilution_conc, how='left', 
                              on=peptide_dilution_name_col, suffixes=('', '_y'))
df_area_ratio_conc.drop(df_area_ratio_conc.filter(regex='_y$').columns, axis=1, inplace=True)

# def get_rep_plot_cat(rep_val:str):
#     """Get the plot category (C18_SDS, C18_SDC, etc..)."""

#     select_elements = rep_val.split('_')[1:3]

#     cat = '_'.join(select_elements)
#     return cat

def get_rep_plot_cat(row):
    """Get the plot category (C18_SDS, C18_SDC, etc..)."""

    replicate = row['replicate']
    peptide = row['peptide']
    fragment_ion = row['fragment ion']

    rep_elements = replicate.split('_')[0:3]

    cat_name = [peptide] + [fragment_ion] + rep_elements
    cat_name = '_'.join(cat_name)
    return cat_name

# create plot categories
df_area_ratio_conc['plot_cat'] = df_area_ratio_conc.apply(get_rep_plot_cat, axis=1)
df_area_ratio_conc.head()
# print(df_area_ratio_conc.isna().sum())

,peptide,protein,replicate,product charge,fragment ion,area_ratio,area_min,area_max,peptide_dilution_name,heavy_conc,plot_cat
0,AEFVEVTK,sp|P02769|ALBU_BOVIN,Soroush_C18_SDC_D2_column12_2uL_11,1,b2,4.785271,425566400.0,2.036451e+09,AEFVEVTK_D2,86.5,AEFVEVTK_b2_Soroush_C18_SDC
1,AEFVEVTK,sp|P02769|ALBU_BOVIN,Soroush_C18_SDC_D2_column12_2uL_11,1,y2,5.214290,52236184.0,2.723746e+08,AEFVEVTK_D2,86.5,AEFVEVTK_y2_Soroush_C18_SDC
2,AEFVEVTK,sp|P02769|ALBU_BOVIN,Soroush_C18_SDC_D2_column12_2uL_11,1,y4,4.992701,92338456.0,4.610183e+08,AEFVEVTK_D2,86.5,AEFVEVTK_y4_Soroush_C18_SDC
3,AEFVEVTK,sp|P02769|ALBU_BOVIN,Soroush_C18_SDC_D2_column12_2uL_11,1,y5,5.242521,117168584.0,6.142588e+08,AEFVEVTK_D2,86.5,AEFVEVTK_y5_Soroush_C18_SDC
4,AEFVEVTK,sp|P02769|ALBU_BOVIN,Soroush_C18_SDC_D2_column12_2uL_11,1,y6,5.069243,377321984.0,1.912737e+09,AEFVEVTK_D2,86.5,AEFVEVTK_y6_Soroush_C18_SDC


## Get Linear Fit parameters

In [8]:
from sklearn import linear_model, metrics
import matplotlib.pyplot as plt

def get_linear_fit(data:pd.DataFrame, plot_cat:str):

    cat_data = data[data['plot_cat'] == plot_cat]
    # print(cat_data)

    # fig, ax = plt.subplots()

    x = cat_data['area_ratio'].values
    y = cat_data['heavy_conc'].values

    # ax.scatter(x, y)

    x = x[:, np.newaxis]
    y = y[:, np.newaxis]

    model = linear_model.LinearRegression()

    model.fit(x, y)

    y_fit = model.predict(x)

    # ax.plot(x.flatten(), y_fit.flatten())

    r2 = metrics.r2_score(y, y_fit)
    intercept = model.intercept_.squeeze()
    grad = model.coef_[0].squeeze()


    # print(f"R^2: {r2}")
    # print(f"intercept: {intercept}")
    # print(f"gradient:{grad}")

    # plt.show()
    
    return (r2, intercept, grad)

plot_cats = df_area_ratio_conc['plot_cat'].unique()

# plot_df = df_area_ratio_conc[df_area_ratio_conc.peptide=='GGLEPINFQTAADQAR'] # get only one peptide
# plot_df = plot_df[[peptide_dilution_name_col, 'area_ratio', 'heavy_conc', 'plot_cat']]
plot_df = df_area_ratio_conc


cats = []
r2s = []
intercepts = []
grads = []

for cat in plot_cats:
    # print(cat)
    r2, intercept, grad = get_linear_fit(plot_df, cat)
    cats.append(cat)
    r2s.append(r2)
    intercepts.append(intercept)
    grads.append(grad)
    



df_plot_cat_fit_params = pd.DataFrame({'plot_cat':cats,
              'R2':r2s,
              'intercept':intercepts,
              'gradient':grads})




In [9]:
df_area_ratio_conc_n_fit_params = pd.merge(df_area_ratio_conc,
                                           right=df_plot_cat_fit_params,
                                           on='plot_cat',
                                           how='left')

df_area_ratio_conc_n_fit_params
# save to csv file
df_area_ratio_conc_n_fit_params.to_csv('area_ratio_conc_n_fit_params.csv', index=False)
    

# Fit metric

In [20]:
def get_fit_param_cat(row):
    """Get the fit param category (C18_SDS, C18_SDC, etc..)."""

    replicate = row['replicate']
    peptide = row['peptide']

    rep_elements = replicate.split('_')[0:3]

    cat_name = [peptide]  + rep_elements
    cat_name = '_'.join(cat_name)
    return cat_name

def qtest(data, right=True):

    sorted_data = sorted(data)

    if right:
        gap = sorted_data[-1] - sorted_data[-2]

    else:
        gap = sorted_data[1] - sorted_data[0]

    try:
        range = sorted_data[-1] - sorted_data[0]
        q_val = gap / range
    except ZeroDivisionError:
        q_val = gap

    return q_val

def get_grp_fit_agg(grouped_df:pd.DataFrame):
    """Get the aggregate values for each group."""

    df = grouped_df[['intercept', 'gradient', 'R2']].copy()
    gradient = df['gradient'].values
    mean_grad = gradient.mean()
    stdv_grad = gradient.std()
    cov_grad = stdv_grad / mean_grad
    qtest_r2_right = qtest(df['R2'].values, right=True)
    qtest_int_right = qtest(df['intercept'].values, right=True)

    cols_dict = {'mean_grad':mean_grad, 'stdv_grad':stdv_grad,
                 'cov_grad':cov_grad, 'qtest_r2_right':qtest_r2_right,
                 'qtest_intercept':qtest_int_right}

    return pd.Series(cols_dict)

# create plot categories
temp = df_area_ratio_conc_n_fit_params.copy()
temp['fit_param_grp'] = temp.apply(get_fit_param_cat, axis=1)

cols_to_group_by = ['fit_param_grp']
df_fit_param_agg = temp.groupby(cols_to_group_by).apply(get_grp_fit_agg).reset_index()

df_w_fit_param_agg = pd.merge(left=temp, right=df_fit_param_agg, how='left', on=cols_to_group_by)


,peptide,protein,replicate,product charge,fragment ion,area_ratio,area_min,area_max,peptide_dilution_name,heavy_conc,plot_cat,R2,intercept,gradient,fit_param_grp,mean_grad,stdv_grad,cov_grad,qtest_r2_right,qtest_intercept
0,AEFVEVTK,sp|P02769|ALBU_BOVIN,Soroush_C18_SDC_D2_column12_2uL_11,1,b2,4.785271,425566400.0,2.036451e+09,AEFVEVTK_D2,86.5,AEFVEVTK_b2_Soroush_C18_SDC,0.999321,-1.1851182430159426,18.288988526639038,AEFVEVTK_Soroush_C18_SDC,17.760769,0.890032,0.050112,0.0,0.0
1,AEFVEVTK,sp|P02769|ALBU_BOVIN,Soroush_C18_SDC_D2_column12_2uL_11,1,y2,5.214290,52236184.0,2.723746e+08,AEFVEVTK_D2,86.5,AEFVEVTK_y2_Soroush_C18_SDC,0.999455,-1.0386297514165719,16.760133904180453,AEFVEVTK_Soroush_C18_SDC,17.760769,0.890032,0.050112,0.0,0.0
2,AEFVEVTK,sp|P02769|ALBU_BOVIN,Soroush_C18_SDC_D2_column12_2uL_11,1,y4,4.992701,92338456.0,4.610183e+08,AEFVEVTK_D2,86.5,AEFVEVTK_y4_Soroush_C18_SDC,0.999336,-1.4140921074358346,17.575471055966215,AEFVEVTK_Soroush_C18_SDC,17.760769,0.890032,0.050112,0.0,0.0
3,AEFVEVTK,sp|P02769|ALBU_BOVIN,Soroush_C18_SDC_D2_column12_2uL_11,1,y5,5.242521,117168584.0,6.142588e+08,AEFVEVTK_D2,86.5,AEFVEVTK_y5_Soroush_C18_SDC,0.999419,-1.0674445664184518,16.67428663220671,AEFVEVTK_Soroush_C18_SDC,17.760769,0.890032,0.050112,0.0,0.0
4,AEFVEVTK,sp|P02769|ALBU_BOVIN,Soroush_C18_SDC_D2_column12_2uL_11,1,y6,5.069243,377321984.0,1.912737e+09,AEFVEVTK_D2,86.5,AEFVEVTK_y6_Soroush_C18_SDC,0.999375,-1.1161402670400307,17.309569164464367,AEFVEVTK_Soroush_C18_SDC,17.760769,0.890032,0.050112,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,YICDNQDTISSK,sp|P02769|ALBU_BOVIN,Soroush_MCX_SDC_column12_2uL_23,1,y9,0.000000,4251090.0,0.000000e+00,YICDNQDTISSK_D0,0.0,YICDNQDTISSK_y9_Soroush_MCX_SDC,0.995006,-4.687844501641244,2.0857505103374714,YICDNQDTISSK_Soroush_MCX_SDC,2.076733,0.136740,0.065844,0.0,0.0
266,YICDNQDTISSK,sp|P02769|ALBU_BOVIN,Soroush_MCX_SDC_column12_2uL_23,2,precursor,0.001054,99149248.0,1.045240e+05,YICDNQDTISSK_D0,0.0,YICDNQDTISSK_precursor_Soroush_MCX_SDC,0.997090,-3.6287598335339766,1.7686713047091316,YICDNQDTISSK_Soroush_MCX_SDC,2.076733,0.136740,0.065844,0.0,0.0
267,YICDNQDTISSK,sp|P02769|ALBU_BOVIN,Soroush_MCX_SDC_column12_2uL_23,2,precursor [M+1],0.000000,71641152.0,0.000000e+00,YICDNQDTISSK_D0,0.0,YICDNQDTISSK_precursor [M+1]_Soroush_MCX_SDC,0.997094,-3.624696880648294,1.9872262295651868,YICDNQDTISSK_Soroush_MCX_SDC,2.076733,0.136740,0.065844,0.0,0.0
268,YICDNQDTISSK,sp|P02769|ALBU_BOVIN,Soroush_MCX_SDC_column12_2uL_23,2,precursor [M+2],0.010409,32146408.0,3.346110e+05,YICDNQDTISSK_D0,0.0,YICDNQDTISSK_precursor [M+2]_Soroush_MCX_SDC,0.997215,-3.5729942042394924,2.0797923741269617,YICDNQDTISSK_Soroush_MCX_SDC,2.076733,0.136740,0.065844,0.0,0.0


In [24]:
df_w_fit_param_agg.to_csv('data_with_fit_param_aggregate.csv', index=False)

In [11]:
def dixon_test(data, left=True, right=True, q_dict=Q95):
    """
    Keyword arguments:
        data = A ordered or unordered list of data points (int or float).
        left = Q-test of minimum value in the ordered list if True.
        right = Q-test of maximum value in the ordered list if True.
        q_dict = A dictionary of Q-values for a given confidence level,
            where the dict. keys are sample sizes N, and the associated values
            are the corresponding critical Q values. E.g.,
            {3: 0.97, 4: 0.829, 5: 0.71, 6: 0.625, ...}

    Returns a list of 2 values for the outliers, or None.
    E.g.,
       for [1,1,1] -> [None, None]
       for [5,1,1] -> [None, 5]
       for [5,1,5] -> [1, None]

    """
    assert(left or right), 'At least one of the variables, `left` or `right`, must be True.'
    assert(len(data) >= 3), 'At least 3 data points are required'
    assert(len(data) <= max(q_dict.keys())), 'Sample size too large'

    sdata = sorted(data)
    Q_mindiff, Q_maxdiff = (0,0), (0,0)

    if left:
        Q_min = (sdata[1] - sdata[0])
        try:
            Q_min /= (sdata[-1] - sdata[0])
        except ZeroDivisionError:
            pass
        Q_mindiff = (Q_min - q_dict[len(data)], sdata[0])

    if right:
        Q_max = abs((sdata[-2] - sdata[-1]))
        try:
            Q_max /= abs((sdata[0] - sdata[-1]))
        except ZeroDivisionError:
            pass
        Q_maxdiff = (Q_max - q_dict[len(data)], sdata[-1])

    if not Q_mindiff[0] > 0 and not Q_maxdiff[0] > 0:
        outliers = [None, None]

    elif Q_mindiff[0] == Q_maxdiff[0]:
        outliers = [Q_mindiff[1], Q_maxdiff[1]]

    elif Q_mindiff[0] > Q_maxdiff[0]:
        outliers = [Q_mindiff[1], None]

    else:
        outliers = [None, Q_maxdiff[1]]
    
        

    return outliers

NameError: name 'Q95' is not defined